# Comment Toxicity Severity Rater
- group by the same comment and find out if that comment is more often rated as more/ less toxic 
- transform the input data to be more / less toxic 0 and 1 // train on these


In [ ]:
#!pip install alt-profanity-check
!pip install joblib
!pip install scikit-learn
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor


In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random


In [2]:
# remove special characters using the regular expression library
# updated with . and = 
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\*)|(\=\=)|(\~) | (\=) | (\.\.\.) |(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile(" (<br\s/><br\s/?)| (\n\n) | (\n) |(\.) |(-)|(/)|(:). ")

#re.escape (* )

# re.sub() // natalie uses this more often

# regex has a method to escape the char properly 
re.escape


# gridsearchcv() // can be slow 
if have a few val, can use 

# use randomsearch() // possible range // faster 

In [3]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [4]:
original_data = pd.read_csv("./data/validation_data.csv")

data_to_test = pd.read_csv('./data/comments_to_score.csv')

In [5]:
pd.set_option('display.max_columns', None)

original_data.head()



,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [6]:
#original_data.iloc[0]
original_data['more_toxic'][0]

'WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=\nWHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back'

In [7]:
cleaned_data = original_data
cleaned_data

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [8]:
clean_less_toxic = clean_tweets(original_data["less_toxic"])
cleaned_data['less_toxic'] = pd.DataFrame(clean_less_toxic)
#cleaned_data

In [9]:
clean_more_toxic = clean_tweets(original_data["more_toxic"])
cleaned_data['more_toxic'] = pd.DataFrame(clean_more_toxic)
cleaned_data

,worker,less_toxic,more_toxic
0,313,this article sucks woo woo wooooooo,what=wher is your sexy pic gone from your main...
1,188,and yes people should recognize that but they ...,daphne guinness top of the mornin my favourite...
2,82,western mediayup because every crime in the en...,atom you dont believe actual photos of masturb...
3,347,and you removed it you numbskull i dont care w...,you seem to have sand in your vagina.might wan...
4,539,smelly vagina bluerasberry why dont you be a m...,hey way to support nazis you racist
...,...,...,...
30103,461,im sorry im not an admin i will give you three...,get out my large penis
30104,527,im sorry im not an admin i will give you three...,get out my large penis
30105,352,wow...are you out of your mind how was my edit...,piss off you slant eyed gook
30106,311,wow...are you out of your mind how was my edit...,piss off you slant eyed gook


In [10]:
 cleaned_data['more_toxic'][0]

'what=wher is your sexy pic gone from your main page put it back'

In [11]:
cleaned_data_shuffle = pd.DataFrame()
indices_moreToxic = []
col_1 = []
col_0 = []
random.seed(0)

for i in range(cleaned_data.shape[0]):
    idx = round(random.uniform(0, 1))
    #print('index of more toxic',idx)
    
    indices_moreToxic.append(idx)#index of where the more toxic comment is 
        #1 for right column, 0 for left column 
    
    if idx == 1: 
        col_1.append( cleaned_data['more_toxic'][i])
        col_0.append( cleaned_data['less_toxic'][i])
        
    else:
        col_0.append(cleaned_data['more_toxic'][i])
        col_1.append( cleaned_data['less_toxic'][i])


cleaned_data_shuffle['col_0'] = pd.Series(col_0)
#trans = pd.DataFrame(comment, columns = ['comment']) // another method of setting to df col

cleaned_data_shuffle['col_1'] = pd.Series(col_1)
cleaned_data_shuffle['more_toxic_col_idx'] = pd.DataFrame(indices_moreToxic)

cleaned_data_shuffle


,col_0,col_1,more_toxic_col_idx
0,this article sucks woo woo wooooooo,what=wher is your sexy pic gone from your main...,1
1,and yes people should recognize that but they ...,daphne guinness top of the mornin my favourite...,1
2,atom you dont believe actual photos of masturb...,western mediayup because every crime in the en...,0
3,you seem to have sand in your vagina.might wan...,and you removed it you numbskull i dont care w...,0
4,smelly vagina bluerasberry why dont you be a m...,hey way to support nazis you racist,1
...,...,...,...
30103,get out my large penis,im sorry im not an admin i will give you three...,0
30104,im sorry im not an admin i will give you three...,get out my large penis,1
30105,piss off you slant eyed gook,wow...are you out of your mind how was my edit...,0
30106,piss off you slant eyed gook,wow...are you out of your mind how was my edit...,0


In [12]:
type(cleaned_data_shuffle.col_0.values)

numpy.ndarray

In [13]:
#clean the input for predition data
cleaned_text = clean_tweets(data_to_test['text'])

data_to_test['text'] = pd.DataFrame(cleaned_text)
#data_to_test

In [14]:
# extract the labels from the train data
#y = trans.score.values
#X = trans.comment.values

from sklearn.feature_extraction.text import CountVectorizer

# initalize vectorizer: vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english') #stop_words removed: a, the

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(cleaned_data_shuffle.col_0.values) + 
               list(cleaned_data_shuffle.col_1.values) + list(data_to_test.text.values))
    

# transform documents to document-term matrix
col0_vec = vectorizer.transform(cleaned_data_shuffle.col_0)
col1_vec = vectorizer.transform(cleaned_data_shuffle.col_1)
comments_2_score = vectorizer.transform(data_to_test["text"].values)

# combine col0 and col1
col0_1_vec = vectorizer.transform(cleaned_data_shuffle.col_0 + ' ' + cleaned_data_shuffle.col_1)


# for testing on the original clean more and less toxic col (unshuffled)
more_tox_vec = vectorizer.transform(cleaned_data['more_toxic'].values)
less_tox_vec = vectorizer.transform(cleaned_data['less_toxic'].values)

In [62]:
empty_vec = vectorizer.transform([' ']*30108)

In [63]:
empty_vec

<30108x46974 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
#input_features = np.hstack((col0_vec, col1_vec)) # not the right type of concatenation
#input_features

In [15]:
import scipy.sparse as sp

combined_2 = sp.hstack([col0_vec, col1_vec], format='csr')
combined_2

<30108x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 1508591 stored elements in Compressed Sparse Row format>

In [ ]:
input_features

In [ ]:
6 * int(X[0].shape[0] /10)

In [ ]:
6*int(X[0].shape[0]/10)+ int(X[0].shape[0]/5)

In [ ]:
int(X[0].shape[0]/5)

In [ ]:
X[0][0:20]

In [ ]:
x_val = X[:][6 * int(X[0].shape[0]/10) : 6 * int(X[0].shape[0]/10) + int(X[0].shape[0]/5),]
x_val

In [ ]:
x_train = X[:][:][:][1]
x_train


from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = cleaned_data_shuffle.more_toxic_col_idx.values
X = input_features


#valid
x_val = X[:][6 * int(X[0].shape[0] /10):6 * int(X[0].shape[0]/10)+ int(X[0].shape[0]/5),]
y_val = y[6*int(y.shape[0]/10):6*int(y.shape[0]/10) +
          int(y.shape[0]/5),]
# train
x_train = X[:][:6 * int(X[0].shape[0] /10),]
y_train = y[:6 * int(y.shape[0] /10),]



#test
x_test = X[:][6 * int(X[0].shape[0]/10)+ int(X[0].shape[0]/5):,]
y_test = y[6*int(y.shape[0]/10) +
          int(y.shape[0]/5):,]




In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_val.shape

In [ ]:
y_val.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

# concatenate on the last dim
# same rows, vec from first one on the sec vec

# TODO: LOOK AT SLACK WITH NATALIE FOR NORMAL MODEL TRAIN 2) LISTEN TO THE PHONE RECORDING FOR LSTM AND LOGISTIC REGRESSION TRAIN 

# 2 features ? 

for part 1 and part 2? 

## concatenate the whole vector count

# but how to tell the model that feature 1 of column 1 is for lesser -- how to put a label on that ? when the same row has another column for more toxic

# comparing the classification accuracy of basic models // should place after the cross-validated

In [67]:
from scipy.stats import rankdata
# make a function to score the prediction and compare if the "lesser" toxic rated is lower prob of rated toxicity
def comp_toxicity (model, less_tox_val_vec, more_tox_val_vec):
    '''
    model is the trained model to be evaluated against the less and more toxic comments
    objective is to see if more toxic comments have a higher score

    less_tox_val_vec and more_tox_val_vec are the respective column in dataframe that contains the comments vectorized
    '''
    result = res = np.zeros((1,2))




    for i in range(cleaned_data.shape[0]):
        result = rankdata([model.predict_proba(less_tox_val_vec[i]).tolist()[0][1], 
                 model.predict_proba(more_tox_val_vec[i]).tolist()[0][1]], method = 'ordinal' )
        result -= 1
        #print(result)
        res += result
        #print(res[0])
        '''
        if i == 5:
            break
        '''
    print ('final result: ',res[0])
    return res[0]



In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=10, random_state=1)
clf.fit(col0_1_vec, cleaned_data_shuffle.more_toxic_col_idx )


In [ ]:
comp_toxicity(clf, less_tox_vec, more_tox_vec)

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=10, random_state=1)
clf.fit(input_features, cleaned_data_shuffle.more_toxic_col_idx )


In [ ]:
comp_toxicity(clf, less_tox_vec, more_tox_vec)

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=3)
clf.fit(col0_vec + col1_vec, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

In [22]:
less_tox_repeat_vec = sp.hstack([less_tox_vec, less_tox_vec], format='csr')
less_tox_repeat_vec

<30108x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 1601398 stored elements in Compressed Sparse Row format>

In [23]:
more_tox_repeat_vec = sp.hstack([more_tox_vec, more_tox_vec], format='csr')
more_tox_repeat_vec

<30108x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 1415784 stored elements in Compressed Sparse Row format>

In [112]:
less_tox_empty_vec = sp.hstack([empty_vec, less_tox_vec], format='csr') 
# need empty_vec on the left side, comp_toxicity assumes more toxic on the right -- toxicity prob score
less_tox_empty_vec

<30108x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 800699 stored elements in Compressed Sparse Row format>

In [113]:
more_tox_empty_vec = sp.hstack([ empty_vec, more_tox_vec], format='csr')
more_tox_empty_vec

<30108x93948 sparse matrix of type '<class 'numpy.int64'>'
	with 707892 stored elements in Compressed Sparse Row format>

In [114]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec , more_tox_empty_vec)

/Users/ericling/Documents/GitHub/DataSci Projects/KaggleToxicity/Kaggle_ToxicityRater/kaggle_kernel/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


final result:  [ 7725. 22383.]


array([ 7725., 22383.])

In [92]:
clf = LogisticRegression()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )


clf.predict_proba(less_tox_empty_vec[:6])

/Users/ericling/Documents/GitHub/DataSci Projects/KaggleToxicity/Kaggle_ToxicityRater/kaggle_kernel/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[0.42712499, 0.57287501],
       [0.61437132, 0.38562868],
       [0.66234533, 0.33765467],
       [0.71174425, 0.28825575],
       [0.39100691, 0.60899309],
       [0.78307489, 0.21692511]])

In [106]:
clf.predict_proba(more_tox_empty_vec[:6]).tolist()

[[0.4284193843862787, 0.5715806156137213],
 [0.553597686196861, 0.44640231380313894],
 [0.8605638169198045, 0.13943618308019542],
 [0.8790218005259434, 0.12097819947405668],
 [0.8164261106112024, 0.1835738893887976],
 [0.4977800759680051, 0.5022199240319949]]

In [115]:
pd.DataFrame(clf.predict(more_tox_empty_vec)).value_counts()

1    22385
0     7723
dtype: int64

In [116]:
pd.DataFrame(clf.predict(less_tox_empty_vec)).value_counts()

0    17811
1    12297
dtype: int64

In [96]:
summed_more_tox_is_more_tox = 0
for i in range(30108):
    summed_more_tox_is_more_tox = clf.predict_proba(more_tox_empty_vec)[i][0].sum()
    

#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_repeat_vec , more_tox_repeat_vec)

clf = LogisticRegression()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )


clf.predict_proba(less_tox_repeat_vec[:6])

clf = LogisticRegression()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )


clf.predict_proba(more_tox_repeat_vec[:6])

In [72]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(combined_2, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_empty_vec, more_tox_empty_vec)

final result:  [21593.  8515.]


array([21593.,  8515.])

In [25]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(col0_1_vec, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

final result:  [15231. 14877.]


array([15231., 14877.])

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(col0_1_vec, cleaned_data_shuffle.more_toxic_col_idx )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

In [ ]:
C = 1.0
clf = svm.SVC(kernel='linear', C=C, probability = True)
clf.fit(x_train_vec,y_train )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

clf = svm.SVC(kernel='rbf', C=C, probability = True)
clf.fit(x_train_vec,y_train )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

clf = svm.SVC(kernel='sigmoid', C=C, probability = True)
clf.fit(x_train_vec,y_train )
comp_toxicity(clf, less_tox_vec, more_tox_vec)

clf = svm.SVC(kernel='poly', C=C, probability = True)
clf.fit(x_train_vec,y_train )
comp_toxicity(clf, less_tox_vec, more_tox_vec)




# cross validation on different models

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


clf = RandomForestClassifier(n_estimators=10, random_state=1)
cv_scores = cross_val_score(clf, x_train_val_vec, y_train_all, cv=10)

print ('random forest: ',cv_scores.mean())

In [ ]:
#KNN choosing best neighbour number 

from sklearn import neighbors


for n in range(1, 15):
    clf = neighbors.KNeighborsClassifier(n_neighbors=n)
    cv_scores = cross_val_score(clf, x_train_val_vec, y_train_all, cv=10)
    print (n, cv_scores.mean())
    
#k = 8 , 3 are first and sec place for best predictor respectively

In [ ]:
#KNN 

from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=8)
cv_scores = cross_val_score(clf, x_train_val_vec, y_train_all, cv=10)

cv_scores.mean()

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
cv_scores = cross_val_score(clf, x_train_val_vec, y_train_all, cv=10)
cv_scores.mean()

In [ ]:
#Naive Bayes
#usually not the best performance, but fast to train
from sklearn import preprocessing

from sklearn.naive_bayes import MultinomialNB

#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.MaxAbsScaler() #don't really need scaling since the input is already binary 
training_inputs_minmax = scaler.fit_transform(x_train_val_vec)

clf = MultinomialNB()
cv_scores = cross_val_score(clf, training_inputs_minmax, y_train_all, cv=10)

print ('minmax scaled Baye\'s: ',cv_scores.mean())

In [ ]:
#SVM with kernel tuning

from sklearn import svm

C = 1.0
svc = svm.SVC(kernel='linear', C=C)
cv_scores = cross_val_score(svc, x_train_val_vec, y_train_all, cv=10)
print ('linear kernel', cv_scores.mean())


svc = svm.SVC(kernel='rbf', C=C)
cv_scores = cross_val_score(svc, x_train_val_vec, y_train_all, cv=10)
cv_scores.mean()
print ('rbf kernel', cv_scores.mean())


svc = svm.SVC(kernel='sigmoid', C=C)
cv_scores = cross_val_score(svc, x_train_val_vec, y_train_all, cv=10)
cv_scores.mean()
print ('sigmoid kernel', cv_scores.mean())

svc = svm.SVC(kernel='poly', C=C)
cv_scores = cross_val_score(svc, x_train_val_vec, y_train_all, cv=10)
cv_scores.mean()
print ('poly kernel', cv_scores.mean())

# try on colab: keras lstm, xgb 

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier( eta = 0.6 ,objective = 'multi:softmax', num_class = 2) #eta higher starts losing acc -- lower is slower
cv_scores = cross_val_score(clf, x_train_val_vec, y_train_all, cv=10)
print (cv_scores.mean())

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index) + 1

sequences_train = tokenizer.texts_to_sequences(x_train)
padded_train = sequence.pad_sequences(sequences_train, padding='post', maxlen = 50) 

sequences_val = tokenizer.texts_to_sequences(x_val)
padded_val = sequence.pad_sequences(sequences_val, padding='post', maxlen = 50) 

In [ ]:
#y_train
#y_val
print(padded_val.shape)
print(padded_train.shape)
padded_val[0]

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128)) 
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) #128 intermediate recurrent neurons in the lstm layer // maybe need more or less
#single layer 25 does better
#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid')) #sigmoid becuz binary classification

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) #optimization

In [ ]:
my_callbacks = [
                EarlyStopping(patience = 3, monitor = 'val_loss'),
                ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5') #edit name: as diff parameter changed
]

# further optimization: https://keras.io/api/callbacks/, https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping#args


In [ ]:
model.fit(padded_train, y_train,
          batch_size=32, #num of data to use per step size
          epochs=15,
          verbose=2,
          validation_data=(padded_val, y_val), callbacks = my_callbacks)

In [ ]:
#best model from callback 
from tensorflow import keras
model = keras.models.load_model('model.02-0.62.h5')

In [ ]:
#pad the test set prediction: 7:35 from https://www.youtube.com/watch?v=Y_hzMnRXjhI


#model prediction on the padded test set 

# accuracy still very bad: consider grabbing other dataset to train OR adding more dense layers with the bigger dataset

## trying with different maxlen in lstm 
was using padding maxlen = 20, 59.87%
maxlen = 80, got about 55%
maxlen = 50, got 55.79%

## adding dense intermediate layers with maxlen = 50 
model.add(Dense(25, activation='relu')) // 61%

//not as good: 60.47%
model.add(Dense(90, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(24, activation='relu'))

//60.15%
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))

//61.46%, 60.08
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))

//60.65%
model.add(Dense(16, activation='relu'))

//60.26%
model.add(Dense(25, activation='relu'))
model.add(Dense(8, activation='relu'))

//59%
model.add(Dense(8, activation='relu'))

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=32,
                            verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)